# Data Wrangling Exercises

### Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2. Let's see how writing to the local disk works in spark:

    * Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    * Inspect your folder structure. What do you notice?
    
3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

<hr style="border:3px solid black"> </hr>

# Part I:

### #1. Read the case, department, and source data into their own spark dataframes.

In [2]:
# Read in CSV file 
case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [3]:
# Read in CSV file 
dept = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [4]:
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

<hr style="border:1px solid black"> </hr>

### #2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json


In [5]:
case.write.json("case_json", mode="overwrite")

In [6]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [7]:
dept.write.json("dept_json", mode="overwrite")

In [8]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [9]:
# write to json

source.write.json("source_json", mode="overwrite")

In [10]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [11]:
# write to csv

source.write.csv("sources_csv", mode="overwrite")

#### Inspect your folder structure. What do you notice?

- new folders created

<hr style="border:1px solid black"> </hr>

### #3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [12]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [13]:
# check dtypes for source dataframe

source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

- source_id should be a numeric value.

In [14]:
source = source.withColumn('source_id', col('source_id').cast('int'))

In [15]:
# check dtypes now

source.dtypes

[('source_id', 'int'), ('source_username', 'string')]

In [16]:
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [17]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [18]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('SLA_due_date', to_timestamp('SLA_due_date', fmt))

In [19]:
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [20]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [21]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [22]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

<hr style="border:3px solid black"> </hr>

# Part II:

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [23]:
open_cases = case.where(case.case_closed == False)

In [24]:
open_cases.select("*").show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014128388           
 case_opened_date     | 2018-01-02 09:39:00  
 case_closed_date     | null                 
 SLA_due_date         | 2018-01-09 09:39:00  
 case_late            | true                 
 num_days_late        | 211.5974884          
 case_closed          | false                
 dept_division        | 311 Call Center      
 service_request_type | Complaint            
 SLA_days             | 7.0                  
 case_status          | Open                 
 source_id            | mt13131              
 request_address      | 7326  WESTGLADE P... 
 council_district     | 6                    
-RECORD 1------------------------------------
 case_id              | 1014128790           
 case_opened_date     | 2018-01-02 10:49:00  
 case_closed_date     | null                 
 SLA_due_date         | 2018-05-10 10:49:00  
 case_late            | true                 
 num_days_late        | 90.5492592

In [25]:
open_cases = open_cases.withColumn('days_past_sla', datediff(current_timestamp(), 'sla_due_date'))\
.withColumn('days_open', datediff(current_timestamp(), 'case_opened_date'))

In [26]:
open_cases.sort(desc('days_past_sla')).show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 days_past_sla        | 1756                 
 days_open            | 1772                 
only showing top 1 row



In [27]:
open_cases.sort(desc('days_open')).show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 SLA_due_date         | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 6                    
 days_past_sla        | 1756                 
 days_open            | 1772                 
only showing top 1 row



<hr style="border:1px solid black"> </hr>

### 2. How many Stray Animal cases are there?
- Answer: 26,760

In [28]:
case.groupBy('service_request_type').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|86855|
|Overgrown Yard/Trash|65895|
|        Bandit Signs|32910|
|        Damaged Cart|30338|
|Front Or Side Yar...|28794|
|        Stray Animal|26760|
|Aggressive Animal...|24882|
|Cart Exchange Req...|22024|
|Junk Vehicle On P...|21473|
|     Pot Hole Repair|20616|
|Alley-Way Mainten...|20214|
|    Lost/Stolen Cart|18731|
|Right Of Way/Side...|17699|
|   Dead Animal - Dog|16714|
|       Cart Delivery|15471|
|   Dead Animal - Cat|14983|
|      Animal Neglect|13441|
|  Dead Animal - Misc|13234|
|Trapped/Confined ...|11354|
|Public Nuisance(O...|10715|
+--------------------+-----+
only showing top 20 rows



In [29]:
case.where(expr("service_request_type = 'Stray Animal'")).count()

26760

<hr style="border:1px solid black"> </hr>

### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [30]:
case.where(case.dept_division == "Field Operations").where(case.service_request_type != "Officer Standby").count()

113902

<hr style="border:1px solid black"> </hr>

### 4.Convert the council_district column to a string column.

In [31]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [32]:
#council_district as a string instead of int
case = case.withColumn('council_district', col('council_district').cast('string'))

In [33]:
#take a look
case.select('council_district').show(4)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
+----------------+
only showing top 4 rows



In [34]:
#make sure it was changed
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

<hr style="border:1px solid black"> </hr>

### 5. Extract the year from the case_closed_date column.

In [35]:
#look at datatypes
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

In [36]:
#need to change dtype to datetime
# to_timestamp, fmt

fmt = "M/d/yy H:mm"
case = case.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))

In [37]:
#create new column with year only
case = case.withColumn('year',year(case.case_closed_date))

#take a look
case.show(2, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 SLA_due_date         | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
 year                 | 2018                                 
-RECORD 

<hr style="border:1px solid black"> </hr>

### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [38]:
case = case.select("*", (case.num_days_late * 24).alias('num_hours_late'))

In [39]:
case.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 year                 | 2018                 
 num_hours_late       | -23964.2102784       
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05

<hr style="border:1px solid black"> </hr>

### 7. Join the case data with the source and department data.

In [40]:
#look at df to find foreign keys
source.show(2, truncate = False, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
only showing top 2 rows



In [41]:
#look at df to find foreign keys
dept.show(2, truncate = False, vertical=True)

-RECORD 0----------------------------------------
 dept_division          | 311 Call Center        
 dept_name              | Customer Service       
 standardized_dept_name | Customer Service       
 dept_subject_to_SLA    | YES                    
-RECORD 1----------------------------------------
 dept_division          | Brush                  
 dept_name              | Solid Waste Management 
 standardized_dept_name | Solid Waste            
 dept_subject_to_SLA    | YES                    
only showing top 2 rows



In [42]:
#look at df to find foreign keys
case.show(1,truncate = False, vertical=True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 SLA_due_date         | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
 year                 | 2018                                 
 num_hou

In [43]:
#no foreign keys in common between source and dept
#join case and dept first
join_df = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
     # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

In [44]:
#used join_df to join with source now
final_join = (
    case
    # left join on dept_division
    .join(source, "source_id", "left")
     # drop all the columns except for standardized name, as it has much fewer unique values
    # convert to a boolean
)

In [45]:
final_join.show(1, False, True)

-RECORD 0----------------------------------------------------
 source_id            | svcCRMLS                             
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 SLA_due_date         | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
 year                 | 2018                                 
 num_hou

<hr style="border:1px solid black"> </hr>

### 8. Are there any cases that do not have a request source?

In [46]:
source.filter('source_id is null').show(5, False, True)

-RECORD 0------------------------------
 source_id       | null                
 source_username | Blanca Rosa Wilkins 
-RECORD 1------------------------------
 source_id       | null                
 source_username | Bianca Maldonado    
-RECORD 2------------------------------
 source_id       | null                
 source_username | Christine Gutierrez 
-RECORD 3------------------------------
 source_id       | null                
 source_username | CRM_Listener        
-RECORD 4------------------------------
 source_id       | null                
 source_username | Chris Villa         
only showing top 5 rows



<hr style="border:1px solid black"> </hr>

### 9. What are the top 10 service request types in terms of number of requests?

In [47]:
case.groupby('service_request_type').count().sort(desc('count')).show(10, truncate=False)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |86855|
|Overgrown Yard/Trash            |65895|
|Bandit Signs                    |32910|
|Damaged Cart                    |30338|
|Front Or Side Yard Parking      |28794|
|Stray Animal                    |26760|
|Aggressive Animal(Non-Critical) |24882|
|Cart Exchange Request           |22024|
|Junk Vehicle On Private Property|21473|
|Pot Hole Repair                 |20616|
+--------------------------------+-----+
only showing top 10 rows



<hr style="border:1px solid black"> </hr>

### 10. What are the top 10 service request types in terms of average days late?

In [48]:
case.groupby('service_request_type').mean('num_days_late').sort(desc('avg(num_days_late)')).show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420943|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.11255641500003|
|Zoning: Recycle Yard                  |135.92851612479797|
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |77.90021217000002 |
|Complaint                             |72.5179093265971  |
+--------------------------------------+------------------+
only showing top 10 rows



<hr style="border:1px solid black"> </hr>

### 11. Does number of days late depend on department?
- Answer: yes. it does

In [49]:
case.groupby("dept_division").mean('num_days_late').sort(desc('avg(num_days_late)')).show(10, truncate=False)

+-----------------------------+------------------+
|dept_division                |avg(num_days_late)|
+-----------------------------+------------------+
|Code Enforcement (Internal)  |135.92851612479797|
|Reservations                 |66.03116319       |
|311 Call Center              |59.49019459221512 |
|Director's Office Horizontal |36.87389387687182 |
|Engineering Division         |13.148054014077815|
|Shops                        |10.114974371919642|
|Tree Crew                    |4.745302983889023 |
|Solid Waste                  |3.541793412154981 |
|Trades                       |3.1521799543913454|
|Clean and Green Natural Areas|1.691468919487805 |
+-----------------------------+------------------+
only showing top 10 rows



<hr style="border:1px solid black"> </hr>

### 12. How do number of days late depend on department and request type?

In [50]:
case.groupby('dept_division', 'service_request_type').mean('num_days_late').sort(desc('avg(num_days_late)')).show(10, truncate=False)

+---------------------------+--------------------------------------+------------------+
|dept_division              |service_request_type                  |avg(num_days_late)|
+---------------------------+--------------------------------------+------------------+
|Code Enforcement           |Zoning: Junk Yards                    |175.95636210420943|
|Code Enforcement (IntExp)  |Labeling for Used Mattress            |162.43032902285717|
|Code Enforcement (IntExp)  |Record Keeping of Used Mattresses     |153.99724039428568|
|Code Enforcement (IntExp)  |Signage Requied for Sale of Used Mattr|151.63868055333333|
|Code Enforcement (IntExp)  |Storage of Used Mattress              |142.11255641500003|
|Code Enforcement (Internal)|Zoning: Recycle Yard                  |135.92851612479797|
|Code Enforcement           |Donation Container Enforcement        |131.75610506358706|
|Code Enforcement (IntExp)  |License Requied Used Mattress Sales   |128.79828704142858|
|Code Enforcement (IntExp)  |Use